# **Import delle librerie**
Qui importiamo NumPy per il caricamento dei .npy, Pathlib per gestire i percorsi, scikit-learn per l’MLP e la grid-search, e Matplotlib/Seaborn per la confusion matrix.

In [1]:
# Librerie di base
import numpy as np
from pathlib import Path

# Pre-processing e modello
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

# Model selection
from sklearn.model_selection import GridSearchCV

# Metriche e visualizzazione
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# **Configurazione del percorso dati**
DATA_DIR punta alla directory contenente i sei file x_*.npy e y_*.npy. L’assert ti segnala subito se qualcosa non è stato caricato.

In [3]:
# Se i file .npy sono nella cartella 'dataset' del Colab:
DATA_DIR = Path('https://drive.google.com/drive/folders/1ZGSD-7bI77imsKusnm5NzbVoChJsh9e8?usp=drive_link')

# Opzionale: verifica che i file esistano
for fname in ["x_train.npy","y_train.npy","x_val.npy","y_val.npy","x_test.npy","y_test.npy"]:
    assert (DATA_DIR/fname).exists(), f"File mancante: {fname}"
print("Tutti i file .npy sono presenti in", DATA_DIR)

AssertionError: File mancante: x_train.npy

# **Caricamento dei dati**
Verifichiamo che le shape siano coerenti: ad es. X_train dovrebbe essere (N_train, 32, 32, 3) o simile.

In [4]:
# 1. Carica i file .npy in array NumPy
X_train = np.load(DATA_DIR/"x_train.npy")
y_train = np.load(DATA_DIR/"y_train.npy")
X_val   = np.load(DATA_DIR/"x_val.npy")
y_val   = np.load(DATA_DIR/"y_val.npy")
X_test  = np.load(DATA_DIR/"x_test.npy")
y_test  = np.load(DATA_DIR/"y_test.npy")

print("Shape delle matrici:")
print(" X_train:", X_train.shape, " y_train:", y_train.shape)
print(" X_val:  ",   X_val.shape,   " y_val:  ",   y_val.shape)
print(" X_test: ",  X_test.shape,  " y_test: ",  y_test.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'https:/drive.google.com/drive/folders/1ZGSD-7bI77imsKusnm5NzbVoChJsh9e8?usp=drive_link/x_train.npy'

# **Flatten e normalizzazione**
Gli MLP di scikit-learn richiedono input 2D (n_samples, n_features): qui 32×32×3 = 3072 feature per immagine.

In [ ]:
def flatten_and_norm(X: np.ndarray) -> np.ndarray:
    """
    Converte X in float32, normalizza i pixel in [0,1]
    e appiattisce in shape (N, 3072).
    """
    X = X.astype("float32") / 255.0
    return X.reshape(X.shape[0], -1)

X_train = flatten_and_norm(X_train)
X_val   = flatten_and_norm(X_val)
X_test  = flatten_and_norm(X_test)

print("Dopo flatten:", X_train.shape)

# **Standardizzazione**
Portare media a 0 e varianza a 1 migliora la convergenza degli MLP.

In [ ]:
# Fitting dello scaler **solo** sul train, poi applicato a val/test
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val   = scaler.transform(X_val)
X_test  = scaler.transform(X_test)

# **Definizione del modello e griglia di iper-parametri**
- max_iter: epoche massime; - early_stopping: interrompe se la validazione interna non migliora;
- alpha: coopta il weight-decay (L2);
- hidden_layer_sizes: numero di neuroni nei layer.

In [ ]:
# 1) Definisco un MLP "di base" con early stopping
base_mlp = MLPClassifier(
    max_iter=200,
    early_stopping=True,
    n_iter_no_change=15,
    random_state=0
)

# 2) Griglia di parametri da esplorare
param_grid = {
    "hidden_layer_sizes": [(256,), (512, 256)],
    "alpha":                [1e-4, 1e-3],       # regolarizzazione L2
    "learning_rate_init":   [1e-3, 5e-4],       # tasso di apprendimento iniziale
}

# **Grid Search con cross-validation**
GridSearchCV esplora tutte le combinazioni, esegue cross-validation, valuta con accuracy e infine ri‐addestra il modello migliore sull’intero X_train.

In [ ]:
search = GridSearchCV(
    estimator=base_mlp,
    param_grid=param_grid,
    scoring="accuracy",
    cv=3,            # 3‐fold CV sul training set
    n_jobs=-1,       # usa tutti i core
    verbose=2,       # output di avanzamento
    refit=True       # ri-addestra il migliore su tutto il train
)

print("Avvio Grid Search...")
search.fit(X_train, y_train)

print("\nMigliori iper-parametri:")
print(search.best_params_)
print(f"Accuracy media CV: {search.best_score_:.3f}")

# **Valutazione sul validation set “esterno”**
Una volta scelti i parametri, verifichiamo le prestazioni sullo split di validation che non è stato toccato dalla CV.

In [ ]:
# Previsione sulla validation “ufficiale”
val_pred = search.predict(X_val)
val_acc  = accuracy_score(y_val, val_pred)
print(f"Accuracy su validation esterno: {val_acc:.3f}")

# **Valutazione sul test set**
Qui misuriamo accuracy, precision, recall e F1 su dati completamente “nuovi”.

In [ ]:
test_pred = search.predict(X_test)
test_acc  = accuracy_score(y_test, test_pred)
print(f"Accuracy su test set: {test_acc:.3f}\n")
print("Classification report:")
print(classification_report(y_test, test_pred, digits=3))

# **Confusion Matrix**
La confusion matrix aiuta a vedere quali classi vengono confuse più frequentemente.

In [ ]:
cm = confusion_matrix(y_test, test_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix – Test Set")
plt.xlabel("Predetto")
plt.ylabel("Reale")
plt.tight_layout()
plt.show()